### Import Packages

In [ ]:
from sklearn import svm, metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import random

### Load Dataset

In [ ]:
# Import Gaussian Small Dataset
data = np.loadtxt('/Users/elleemortensen/Documents/GitHub/BP24/Ellee/Data/gaussian_small_d_1.tex')
array = np.array(data)
df = pd.DataFrame(array)

### Set-Up and Iterations

In [ ]:
# Set a seed for reproducibility
np.random.seed(42)

# Assume df is your DataFrame
totalrows = df.shape[0]
totalcolumns = 35

# Sample 500 rows from the DataFrame for consistent labels
testing_indices = np.random.permutation(totalrows)-1
sampled_df = df.iloc[testing_indices, :]

# Initialize an empty list to store F1 scores
f1_scores = []

# Initialize the SVM classifier with a linear kernel
clf = svm.SVC(kernel='linear')

# Maximum iterations allowed for sampling and evaluation
max_iterations = 100

# Initialize an array to store all F1 scores across iterations and samples
allf1scores = np.zeros((totalcolumns, max_iterations))

# Iterate over different numbers of sampled columns
for i in range(5, totalcolumns):
    iterations = 0
    # Sample the first 10*(i+1) rows for each iteration
    sampled_rows = sampled_df.iloc[:10 * (i + 1), :]
    while iterations < max_iterations:
        # Randomly sample (i+1) columns from the first 10*(i+1) rows (excluding the target column)
        sampled_columns = sampled_rows.drop(columns=[totalcolumns - 1]).sample(n=i + 1, axis='columns').columns
        
        # Split the dataset into training and testing sets using the sampled indices
        X_train = sampled_rows[sampled_columns]
        y_train = sampled_rows.iloc[:, -1]
        
        # Determine the test index
        test_index = np.ceil((10 * (i+1)) / 0.7).astype(int)
        X_test = sampled_df.iloc[10 * (i + 1):test_index, sampled_columns]
        y_test = sampled_df.iloc[10 * (i + 1):test_index, -1]
        
        # Train the SVM classifier using the training sets for the sampled columns
        clf.fit(X_train, y_train)
        
        # Predict the response for the test dataset
        y_pred = clf.predict(X_test)
        
        # Calculate the F1 score and store it in the allf1scores matrix
        f1 = metrics.f1_score(y_test, y_pred)
        allf1scores[i, iterations] = f1

        # Save the training and testing data at a specific condition for further analysis
        if i == 11 and iterations == 30:
            saved_X_train = X_train.copy()
            saved_y_train = y_train.copy()
            saved_X_test = X_test.copy()
            saved_y_test = y_test.copy()
        
        # Increment the iterations counter
        iterations += 1

In [ ]:
stacked_x = np.vstack((cat_X_train, cat_X_test))
stacked_y = np.vstack((np.array(saved_y_train)[:,np.newaxis], np.array(saved_y_test)[:,np.newaxis]))

print(stacked_x.shape)
print(stacked_y.shape)
print(stacked_all.shape)

stacked_all = np.hstack((stacked_x, stacked_y))
print(stacked_all.shape)

In [ ]:
# Select 4 random columns
random_columns = [101, 125, 8, 139]
print("Randomly selected columns:", random_columns)

# Create a copy of the saved_X_train variable
cat_X_train = saved_X_train.copy()

# Process the randomly selected columns into categorical
for col in random_columns:
    cat_X_train[col] = saved_X_train[col].round().astype(int).astype("category")

# Check if 4 columns were converted to categorical
print(cat_X_train.dtypes)

In [ ]:
# Create a copy of the saved_X_test variable
cat_X_test = saved_X_test.copy()

# Process the randomly selected columns
for col in random_columns:
    cat_X_test[col] = saved_X_test[col].round().astype(int).astype("category")

# Check if 4 columns were converted to categorical
print(cat_X_test.dtypes)
#np.savetxt('gaussmall.csv', stacked_all, delimiter=',')
print(stacked_all.shape)

In [ ]:
# Fit the SVM classifier using the training data with categorical features
clf.fit(cat_X_train, saved_y_train)

# Predict the response for the test dataset
y_pred = clf.predict(cat_X_test)

# Calculate the F1 score for the predictions
f1 = metrics.f1_score(saved_y_test, y_pred)

# Find the F1 score and check the shapes of the datasets
print(f1)
print(cat_X_train.shape)
print(cat_X_test.shape)
print(saved_y_train)
print(saved_y_test)

In [ ]:
# Plotting the F1 scores across iterations
plt.figure(figsize=(10, 6))
plt.plot(range(5, totalcolumns+1), np.min(allf1scores[4:,:], axis=1), marker='o', linestyle='-', color='b', label='F1 Score')
plt.title('F1 Scores Across Iterations')
plt.xlabel('Number of Columns')
plt.ylabel('Average F1 Score')
plt.xticks(range(1, 37))
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()